# Modeling

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('modeling_data.csv')

In [4]:
data = data.drop(columns=['Unnamed: 0','log_date','log_time'])

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   fare_amount      1000000 non-null  float64
 1   passenger_count  1000000 non-null  int64  
 2   log_hour         1000000 non-null  int64  
 3   log_min          1000000 non-null  int64  
 4   log_year         1000000 non-null  int64  
 5   log_month        1000000 non-null  int64  
 6   log_day          1000000 non-null  int64  
 7   7to15            1000000 non-null  int64  
 8   16to19           1000000 non-null  int64  
 9   20to6            1000000 non-null  int64  
 10  dayofweek        1000000 non-null  int64  
 11  weekday          1000000 non-null  int64  
 12  is_extracharge   1000000 non-null  int64  
 13  taxi_amount      1000000 non-null  int64  
 14  dist             1000000 non-null  float64
dtypes: float64(2), int64(13)
memory usage: 114.4 MB


In [6]:
x_train = data.iloc[:,data.columns!='fare_amount']
y_train = data['fare_amount'].values

In [7]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=0.2,random_state=10)

In [9]:
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(800000, 14)
(200000, 14)
(800000,)
(200000,)


In [10]:
y_train

array([ 7.3 ,  7.7 ,  8.  , ..., 34.83, 25.  , 10.9 ])

lgbm

In [11]:
import lightgbm as lgbm

In [12]:
train_set = lgbm.Dataset(x_train, y_train, silent=True)
train_set

In [13]:
params ={'reg_alpha': 0, 'num_leaves': 80, 'min_child_samples': 15, 'max_depth': 10, 'learning_rate': 0.05}

In [14]:
model_lgbm = lgbm.train(params=params, train_set = train_set, num_boost_round=300)

C:\Users\jisuj\anaconda3\lib\site-packages\lightgbm\basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036338 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 420
[LightGBM] [Info] Number of data points in the train set: 800000, number of used features: 14
[LightGBM] [Info] Start training from score 11.335730


In [33]:
pred_test_y = model_lgbm.predict(x_valid)

In [52]:
#test를 train과 동일하게 전처리
test =  pd.read_csv('for_test.csv')

In [53]:
test.head()

,passenger_count,log_hour,log_min,log_year,log_month,log_day,7to15,16to19,20to6,dayofweek,weekday,is_extracharge,dist,taxi_amount
0,1,13,8,2015,1,27,1,0,0,2,1,0,1094.592705,1
1,1,13,8,2015,1,27,1,0,0,2,1,0,1474.196641,1
2,1,11,53,2011,10,8,1,0,0,6,0,0,357.075676,1
3,1,21,12,2012,12,1,0,0,1,6,0,1,1150.115807,1
4,1,21,12,2012,12,1,0,0,1,6,0,1,2875.803386,1


In [54]:
pred_test_y = model_lgbm.predict(test)

In [79]:
#test를 train과 동일하게 전처리
test_index =  pd.read_csv('sample_submission.csv')

In [80]:
test_index['fare_amount'] = pred_test_y

In [81]:
test_index.head()

,key,fare_amount
0,2015-01-27 13:08:24.0000002,22.054313
1,2015-01-27 13:08:24.0000003,22.054313
2,2011-10-08 11:53:44.0000002,14.360644
3,2012-12-01 21:12:12.0000002,21.584657
4,2012-12-01 21:12:12.0000003,21.584657


In [87]:
test_index.to_csv('sub_1.csv', index=False)

In [88]:
sub =  pd.read_csv('sub_1.csv')

In [89]:
sub

,key,fare_amount
0,2015-01-27 13:08:24.0000002,22.054313
1,2015-01-27 13:08:24.0000003,22.054313
2,2011-10-08 11:53:44.0000002,14.360644
3,2012-12-01 21:12:12.0000002,21.584657
4,2012-12-01 21:12:12.0000003,21.584657
...,...,...
9909,2015-05-10 12:37:51.0000002,17.501945
9910,2015-01-12 17:05:51.0000001,11.115901
9911,2015-04-19 20:44:15.0000001,11.125684
9912,2015-01-31 01:05:19.0000005,11.414613
